In [1]:
pip install gymnasium

Note: you may need to restart the kernel to use updated packages.


In [2]:

import gymnasium 

In [3]:
import gymnasium as gym
env = gym.make("CartPole-v1")

observation, info = env.reset(seed=42)
for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()
env.close()

In [4]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install gymnasium[classic-control]

Note: you may need to restart the kernel to use updated packages.


In [6]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import time

class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.model = QNetwork(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        act_values = self.model(state)
        return np.argmax(act_values.detach().numpy()[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                next_state = torch.FloatTensor(next_state).unsqueeze(0)
                target = reward + self.gamma * torch.max(self.model(next_state)[0]).item()
            state = torch.FloatTensor(state).unsqueeze(0)
            target_f = self.model(state)
            target_f[0][action] = target
            self.optimizer.zero_grad()
            loss = self.criterion(target_f, self.model(state))
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, path):
        self.model.load_state_dict(torch.load(path))

    def save(self, path):
        torch.save(self.model.state_dict(), path)

def run_dqn_agent(env, episodes=1000, batch_size=32):
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    scores = []

    for e in range(episodes):
        state, info = env.reset(seed=42)
        state = np.array(state)
        total_reward = 0
        for time in range(500):
            action = agent.act(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            next_state = np.array(next_state)
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            if done:
                print(f"Episode: {e}/{episodes}, score: {total_reward}, e: {agent.epsilon:.2}")
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        scores.append(total_reward)

    agent.save("dqn_cartpole_model.pth")

    plt.plot(scores)
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.savefig('dqn_training_plot.png')
    plt.close()

    return agent

def test_dqn_agent(env, agent, episodes=5):
    for e in range(episodes):
        state, info = env.reset(seed=42)
        state = np.array(state)
        total_reward = 0
        for t in range(500):
            action = agent.act(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            env.render()
            time.sleep(0.01)
            done = terminated or truncated
            state = np.array(next_state)
            total_reward += reward
            if done:
                print(f"Test Episode: {e}/{episodes}, score: {total_reward}")
                break
    env.close()

if __name__ == "__main__":
    env = gym.make("CartPole-v1", render_mode="human")
    agent = run_dqn_agent(env, episodes=50, batch_size=32)
    test_dqn_agent(env, agent, episodes=5)

Episode: 0/50, score: 18.0, e: 1.0
Episode: 1/50, score: 50.0, e: 0.84
Episode: 2/50, score: 33.0, e: 0.71
Episode: 3/50, score: 18.0, e: 0.66
Episode: 4/50, score: 11.0, e: 0.62
Episode: 5/50, score: 11.0, e: 0.59
Episode: 6/50, score: 23.0, e: 0.53
Episode: 7/50, score: 19.0, e: 0.49
Episode: 8/50, score: 23.0, e: 0.44
Episode: 9/50, score: 20.0, e: 0.4
Episode: 10/50, score: 10.0, e: 0.38
Episode: 11/50, score: 12.0, e: 0.36
Episode: 12/50, score: 12.0, e: 0.34
Episode: 13/50, score: 15.0, e: 0.32
Episode: 14/50, score: 15.0, e: 0.29
Episode: 15/50, score: 10.0, e: 0.28
Episode: 16/50, score: 26.0, e: 0.25
Episode: 17/50, score: 11.0, e: 0.24
Episode: 18/50, score: 8.0, e: 0.23
Episode: 19/50, score: 10.0, e: 0.22
Episode: 20/50, score: 9.0, e: 0.21
Episode: 21/50, score: 12.0, e: 0.2
Episode: 22/50, score: 15.0, e: 0.18
Episode: 23/50, score: 9.0, e: 0.18
Episode: 24/50, score: 8.0, e: 0.17
Episode: 25/50, score: 8.0, e: 0.17
Episode: 26/50, score: 8.0, e: 0.16
Episode: 27/50, scor